# Managing Your Model Registry

## Registering a Model with MLFlow

In [1]:
import mlflow
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Prepare a dataset and train a simple model
X, y = make_classification()
X_train, X_test, y_train, y_test = train_test_split(X, y)
model = RandomForestClassifier().fit(X_train, y_train)

# Setting the MLflow tracking URI and experiment
mlflow.set_tracking_uri('http://127.0.0.1:5556')  # Adjust as necessary
experiment_name = 'MLBook_Experiment4'
mlflow.set_experiment(experiment_name)

# Start an MLflow run and log the model
with mlflow.start_run():
    mlflow.sklearn.log_model(model, "RandomForest")
    run_id = mlflow.active_run().info.run_id

# Register the model in the model registry
model_uri = f"runs:/{run_id}/RandomForest"
mlflow.register_model(model_uri, "RandomForestClassifier")


2024/03/11 02:27:57 INFO mlflow.tracking.fluent: Experiment with name 'MLBook_Experiment4' does not exist. Creating a new experiment.
/home/romila/enc/home/romila/enc/anaconda3/lib/python3.11/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Registered model 'RandomForestClassifier' already exists. Creating a new version of this model...
2024/03/11 02:27:59 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestClassifier, version 11
Created version '11' of model 'RandomForestClassifier'.


<ModelVersion: aliases=[], creation_timestamp=1710124079411, current_stage='None', description='', last_updated_timestamp=1710124079411, name='RandomForestClassifier', run_id='1be6fd20faaf4319b77ffb5c4c66270a', run_link='', source='mlflow-artifacts:/159951838006847811/1be6fd20faaf4319b77ffb5c4c66270a/artifacts/RandomForest', status='READY', status_message='', tags={}, user_id='', version='11'>

## Updating Model Registry

In [2]:
# Assume drift_detection_function() is a function that detects drift
def drift_detection_function(X_test, y_test):
    return True

if drift_detection_function(X_test, y_test):
    # Retrain the model with new data
    new_model = RandomForestClassifier().fit(X_train, y_train)
    
    # Log and register the new model version
    with mlflow.start_run():
        mlflow.sklearn.log_model(new_model, "RandomForest")
        new_run_id = mlflow.active_run().info.run_id
        model_uri = f"runs:/{new_run_id}/RandomForest"
        mlflow.register_model(model_uri, "RandomForestClassifier")
        # Promote this model to staging or production after validation
        client = mlflow.tracking.MlflowClient()
        run_id = mlflow.active_run().info.run_id
        client.transition_model_version_stage(
            name="RandomForestClassifier",
            version=client.get_latest_versions("RandomForestClassifier")[0].version,
            stage="Production"
        )


Registered model 'RandomForestClassifier' already exists. Creating a new version of this model...
2024/03/11 02:28:10 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestClassifier, version 12
Created version '12' of model 'RandomForestClassifier'.
/tmp/ipykernel_3010725/3543076641.py:20: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/2.9.2/model-registry.html#migrating-from-stages
  version=client.get_latest_versions("RandomForestClassifier")[0].version,
/tmp/ipykernel_3010725/3543076641.py:18: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. 

## Managing Model Metadata and Access

In [3]:
# When logging the model, include additional metadata
with mlflow.start_run() as run:
    mlflow.log_params(model.get_params())
    mlflow.log_metrics({'accuracy': accuracy_score(y_test, model.predict(X_test))})
    mlflow.set_tags({'data_version': 'v1', 'model_type': 'RandomForest'})

    mlflow.sklearn.log_model(model, "my_model", registered_model_name="RandomForestClassifier")

# Set up access control
client.update_model_version(
    name="RandomForestClassifier",
    version=1,
    description="This version is for users in the analytics team."
)

# Specify who can transition models to production
client.update_registered_model(
    name="RandomForestClassifier",
    description="Production models can only be updated by the ML team."
)


Registered model 'RandomForestClassifier' already exists. Creating a new version of this model...
2024/03/11 02:28:18 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: RandomForestClassifier, version 13
Created version '13' of model 'RandomForestClassifier'.


<RegisteredModel: aliases={}, creation_timestamp=1709601968135, description='Production models can only be updated by the ML team.', last_updated_timestamp=1710124098156, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1709602725932, current_stage='Production', description='', last_updated_timestamp=1710124090469, name='RandomForestClassifier', run_id='fded16c2848b4d0791a376b290fe2740', run_link='', source='mlflow-artifacts:/311739764785642136/fded16c2848b4d0791a376b290fe2740/artifacts/RandomForest', status='READY', status_message='', tags={}, user_id='', version='4'>,
 <ModelVersion: aliases=[], creation_timestamp=1710124098137, current_stage='None', description='', last_updated_timestamp=1710124098137, name='RandomForestClassifier', run_id='b7d38af7c8f54d8c971f54f54da979f4', run_link='', source='mlflow-artifacts:/159951838006847811/b7d38af7c8f54d8c971f54f54da979f4/artifacts/my_model', status='READY', status_message='', tags={}, user_id='', version='13'>], name='RandomF